In [3]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [4]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [5]:

from pandas.io import sql as psql

In [6]:
from lib.connected_db import PgsqlExecutor

In [7]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [8]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

# Long investment

In [9]:
def get_long_stock_data(year):
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and price > 1 
    order by dcf_ai_score desc
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data

def get_short_stock_data(year):
    """
    You don't want to short pennies stock, you only care when stock > 10
    """
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and price > 10 
    order by dcf_ai_score
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data
    


In [10]:
REPORT = """
**********- Score: {score} - {symbol} - {exchange}: {start_price} on {start_date} - *********
End date: {end_date} - price: {end_price} - min: {min_price} - max: {max_price}
Profit: {perf_profit} - max potential: {perf_potential} 
Average growth rate: {avg_growth}, consistent: {is_consistent}, project growth: {adj_growth}
DCF: {dcf}, market_dcf: {market_dcf}, adjusted_dcf: {adj_dcf}, dcf_to_price : {dcf_to_price}

"""
import numpy

def report_all(year):
    win = 0
    lose = 0
    profits = []
    potentials = []
    data = get_long_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.year_end_price - row.price) * 100 / row.price
            ,1)
        potential = round(
            (row.year_max_price - row.price) * 100 / row.price
            ,1)
        if profit > 0:
            win += 1
        else:
            lose += 1
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% *** -  win: {} %".format(
        numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)
    ))
    return numpy.mean(profits), numpy.mean(potentials)

def report_short_all(year):
    win = lose = 0
    profits = []
    potentials = []
    data = get_short_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.price - row.year_end_price) * 100 / row.price
            ,1)
        potential = round(
            (row.price - row.year_min_price) * 100 / row.price
            ,1)
        if profit > 0:
            win += 1
        else:
            lose += 1
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% *** -  win: {} %".format(
        numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)
    ))
    return numpy.mean(profits), numpy.mean(potentials)


# Including Pennies Stock - 2019

In [11]:
report_all(2019)


**********- Score: 54.20000076293945 - NVIV - NasdaqCM: 6.747006 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 1.65 - min: 1.2248 - max: 10.05001
Profit: -75.5 - max potential: 49.0 
Average growth rate: 0.187, consistent: 0, project growth: 0.001
DCF: 1795.6, market_dcf: 1184.3, adjusted_dcf: 740.1, dcf_to_price : 10869.31



**********- Score: 45.599998474121094 - JAKK - NasdaqGS: 1.03 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 0.7825 - min: 0.311 - max: 1.14
Profit: -24.0 - max potential: 10.7 
Average growth rate: -0.101, consistent: 0, project growth: -0.167
DCF: 550.6, market_dcf: 11.6, adjusted_dcf: 737.0, dcf_to_price : 71453.4



**********- Score: 40.5 - TCON - NasdaqCM: 2.34 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 2.02 - min: 1.09 - max: 4.38
Profit: -13.7 - max potential: 87.2 
Average growth rate: -0.122, consistent: 0, project growth: -0.24
DCF: -167.6, market_dcf: -149.5, adjust

(10.419999999999998, 69.57000000000001)

# 2018

In [12]:
report_all(2018)


**********- Score: 61.20000076293945 - TNP - NYSE: 2.66 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.36 - min: 2.66 - max: 4.36
Profit: 63.9 - max potential: 63.9 
Average growth rate: 0.421, consistent: 0, project growth: 0.105
DCF: 4145.7, market_dcf: 1122.3, adjusted_dcf: 1174.1, dcf_to_price : 44039.1



**********- Score: 55.900001525878906 - NEON - NasdaqCM: 1.66 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 1.94 - min: 1.28 - max: 3.56
Profit: 16.9 - max potential: 114.5 
Average growth rate: 0.206, consistent: 0, project growth: 0.005
DCF: -111.0, market_dcf: -69.3, adjusted_dcf: -45.9, dcf_to_price : -2865.06



**********- Score: 54.5 - AVID - NasdaqGS: 4.75 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 8.58 - min: 4.41 - max: 10.32
Profit: 80.6 - max potential: 117.3 
Average growth rate: -0.715, consistent: 0, project growth: -1.075
DCF: 494.4, market_dcf: 10.0, adjusted_dcf: 1275.9, dcf

(42.80333333333332, 78.75333333333332)

In [ ]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []

for year in range(2009, 20010):
    print (year)
    profit_per, potential_per = report_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)



2009

**********- Score: 45.099998474121094 - NSYS - NasdaqCM: 3.25 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 4.13 - min: 3.07 - max: 4.62
Profit: 27.1 - max potential: 42.2 
Average growth rate: -0.764, consistent: 0, project growth: -1.495
DCF: -53.5, market_dcf: 10.3, adjusted_dcf: -328.1, dcf_to_price : -10195.38



**********- Score: 44.0 - ADES - NasdaqGM: 3.05 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 5.57995 - min: 2.325 - max: 5.57995
Profit: 82.9 - max potential: 82.9 
Average growth rate: -0.625, consistent: 0, project growth: -0.923
DCF: -280.6, market_dcf: -39.5, adjusted_dcf: -648.0, dcf_to_price : -21345.9



**********- Score: 42.5 - IEC - NasdaqGM: 5.2999 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 7.62 - min: 4.15 - max: 7.62
Profit: 43.8 - max potential: 43.8 
Average growth rate: -0.223, consistent: 0, project growth: -0.421
DCF: 1017.2, market_dcf: 585.0, adjusted_dcf: 215


**********- Score: 53.20000076293945 - IOR - AMEX: 3.4999 on 2012-12-31 00:00:00 - *********
End date: 2013-12-31 00:00:00 - price: 4.99 - min: 3.1708 - max: 5.37
Profit: 42.6 - max potential: 53.4 
Average growth rate: 0.79, consistent: 1, project growth: 0.541
DCF: 492.4, market_dcf: 30.1, adjusted_dcf: 222.8, dcf_to_price : 6265.9



**********- Score: 49.099998474121094 - BKNG - NasdaqGS: 620.39 on 2012-12-31 00:00:00 - *********
End date: 2013-12-31 00:00:00 - price: 1162.4 - min: 620.39 - max: 1192.33
Profit: 87.4 - max potential: 92.2 
Average growth rate: 0.395, consistent: 1, project growth: 0.256
DCF: 755.7, market_dcf: 635.0, adjusted_dcf: 687.8, dcf_to_price : 10.87



**********- Score: 48.70000076293945 - ARL - NYSE: 2.73 on 2012-12-31 00:00:00 - *********
End date: 2013-12-31 00:00:00 - price: 5.2499 - min: 2.71 - max: 5.9
Profit: 92.3 - max potential: 116.1 
Average growth rate: -0.712, consistent: 0, project growth: -1.393
DCF: -1564.6, market_dcf: 90.5, adjusted_dcf:


**********- Score: 48.79999923706055 - BDSI - NasdaqGS: 1.75 on 2016-12-31 00:00:00 - *********
End date: 2017-12-29 00:00:00 - price: 2.95 - min: 1.6 - max: 3.5
Profit: 68.6 - max potential: 100.0 
Average growth rate: -0.465, consistent: 0, project growth: -0.878
DCF: -32.2, market_dcf: -8.0, adjusted_dcf: -108.3, dcf_to_price : -6288.57



**********- Score: 45.900001525878906 - KGJI - NasdaqCM: 7.740003 on 2016-12-31 00:00:00 - *********
End date: 2017-12-29 00:00:00 - price: 12.120004 - min: 6.360002 - max: 13.830005
Profit: 56.6 - max potential: 78.7 
Average growth rate: -0.23, consistent: 0, project growth: -0.458
DCF: -85082.0, market_dcf: -47588.5, adjusted_dcf: -198838.1, dcf_to_price : -2569066.7



**********- Score: 44.0 - BLFS - NasdaqCM: 1.6103 on 2016-12-31 00:00:00 - *********
End date: 2017-12-29 00:00:00 - price: 6.0 - min: 1.62 - max: 6.9
Profit: 272.6 - max potential: 328.5 
Average growth rate: -0.299, consistent: 0, project growth: -0.598
DCF: -15.4, market_dcf


**********- Score: 65.30000305175781 - LJPC - NasdaqCM: 5.49 on 2020-06-18 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 5.49 - min: 5.49 - max: 5.49
Profit: 0.0 - max potential: 0.0 
Average growth rate: -0.184, consistent: 0, project growth: -0.28
DCF: -319.4, market_dcf: -216.7, adjusted_dcf: -468.4, dcf_to_price : -8631.88



**********- Score: 56.400001525878906 - RM - NYSE: 18.64 on 2020-06-18 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 18.64 - min: 18.64 - max: 18.64
Profit: 0.0 - max potential: 0.0 
Average growth rate: -0.18, consistent: 0, project growth: -0.286
DCF: -1077.8, market_dcf: -745.8, adjusted_dcf: -1649.6, dcf_to_price : -8949.79



**********- Score: 41.70000076293945 - CPTA - NasdaqGS: 2.93 on 2020-06-18 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 2.93 - min: 2.93 - max: 2.93
Profit: 0.0 - max potential: 0.0 
Average growth rate: -0.057, consistent: 0, project growth: -0.113
DCF: 102.5, market_dcf: 25.6, adjusted_dc

*** Average Profit: nan% , Potential: nan% ***
2025
*** Average Profit: nan% , Potential: nan% ***
2026
*** Average Profit: nan% , Potential: nan% ***
2027
*** Average Profit: nan% , Potential: nan% ***
2028
*** Average Profit: nan% , Potential: nan% ***
2029
*** Average Profit: nan% , Potential: nan% ***
2030
*** Average Profit: nan% , Potential: nan% ***
2031
*** Average Profit: nan% , Potential: nan% ***
2032
*** Average Profit: nan% , Potential: nan% ***
2033
*** Average Profit: nan% , Potential: nan% ***
2034
*** Average Profit: nan% , Potential: nan% ***
2035
*** Average Profit: nan% , Potential: nan% ***
2036
*** Average Profit: nan% , Potential: nan% ***
2037
*** Average Profit: nan% , Potential: nan% ***
2038
*** Average Profit: nan% , Potential: nan% ***
2039
*** Average Profit: nan% , Potential: nan% ***
2040
*** Average Profit: nan% , Potential: nan% ***
2041
*** Average Profit: nan% , Potential: nan% ***
2042
*** Average Profit: nan% , Potential: nan% ***
2043
*** Average 

...

*** Average Profit: nan% , Potential: nan% ***
13613
*** Average Profit: nan% , Potential: nan% ***
13614
*** Average Profit: nan% , Potential: nan% ***
13615
*** Average Profit: nan% , Potential: nan% ***
13616
*** Average Profit: nan% , Potential: nan% ***
13617
*** Average Profit: nan% , Potential: nan% ***
13618
*** Average Profit: nan% , Potential: nan% ***
13619
*** Average Profit: nan% , Potential: nan% ***
13620
*** Average Profit: nan% , Potential: nan% ***
13621
*** Average Profit: nan% , Potential: nan% ***
13622
*** Average Profit: nan% , Potential: nan% ***
13623
*** Average Profit: nan% , Potential: nan% ***
13624
*** Average Profit: nan% , Potential: nan% ***
13625
*** Average Profit: nan% , Potential: nan% ***
13626
*** Average Profit: nan% , Potential: nan% ***
13627
*** Average Profit: nan% , Potential: nan% ***
13628
*** Average Profit: nan% , Potential: nan% ***
13629
*** Average Profit: nan% , Potential: nan% ***
13630
*** Average Profit: nan% , Potential: nan% ***

*** Average Profit: nan% , Potential: nan% ***
13769
*** Average Profit: nan% , Potential: nan% ***
13770
*** Average Profit: nan% , Potential: nan% ***
13771
*** Average Profit: nan% , Potential: nan% ***
13772
*** Average Profit: nan% , Potential: nan% ***
13773
*** Average Profit: nan% , Potential: nan% ***
13774
*** Average Profit: nan% , Potential: nan% ***
13775
*** Average Profit: nan% , Potential: nan% ***
13776
*** Average Profit: nan% , Potential: nan% ***
13777
*** Average Profit: nan% , Potential: nan% ***
13778
*** Average Profit: nan% , Potential: nan% ***
13779
*** Average Profit: nan% , Potential: nan% ***
13780
*** Average Profit: nan% , Potential: nan% ***
13781
*** Average Profit: nan% , Potential: nan% ***
13782
*** Average Profit: nan% , Potential: nan% ***
13783
*** Average Profit: nan% , Potential: nan% ***
13784
*** Average Profit: nan% , Potential: nan% ***
13785
*** Average Profit: nan% , Potential: nan% ***
13786
*** Average Profit: nan% , Potential: nan% ***

*** Average Profit: nan% , Potential: nan% ***
13925
*** Average Profit: nan% , Potential: nan% ***
13926
*** Average Profit: nan% , Potential: nan% ***
13927
*** Average Profit: nan% , Potential: nan% ***
13928
*** Average Profit: nan% , Potential: nan% ***
13929
*** Average Profit: nan% , Potential: nan% ***
13930
*** Average Profit: nan% , Potential: nan% ***
13931
*** Average Profit: nan% , Potential: nan% ***
13932
*** Average Profit: nan% , Potential: nan% ***
13933
*** Average Profit: nan% , Potential: nan% ***
13934
*** Average Profit: nan% , Potential: nan% ***
13935
*** Average Profit: nan% , Potential: nan% ***
13936
*** Average Profit: nan% , Potential: nan% ***
13937
*** Average Profit: nan% , Potential: nan% ***
13938
*** Average Profit: nan% , Potential: nan% ***
13939
*** Average Profit: nan% , Potential: nan% ***
13940
*** Average Profit: nan% , Potential: nan% ***
13941
*** Average Profit: nan% , Potential: nan% ***
13942
*** Average Profit: nan% , Potential: nan% ***

*** Average Profit: nan% , Potential: nan% ***
14081
*** Average Profit: nan% , Potential: nan% ***
14082
*** Average Profit: nan% , Potential: nan% ***
14083
*** Average Profit: nan% , Potential: nan% ***
14084
*** Average Profit: nan% , Potential: nan% ***
14085
*** Average Profit: nan% , Potential: nan% ***
14086
*** Average Profit: nan% , Potential: nan% ***
14087
*** Average Profit: nan% , Potential: nan% ***
14088
*** Average Profit: nan% , Potential: nan% ***
14089
*** Average Profit: nan% , Potential: nan% ***
14090
*** Average Profit: nan% , Potential: nan% ***
14091
*** Average Profit: nan% , Potential: nan% ***
14092
*** Average Profit: nan% , Potential: nan% ***
14093
*** Average Profit: nan% , Potential: nan% ***
14094
*** Average Profit: nan% , Potential: nan% ***
14095
*** Average Profit: nan% , Potential: nan% ***
14096
*** Average Profit: nan% , Potential: nan% ***
14097
*** Average Profit: nan% , Potential: nan% ***
14098
*** Average Profit: nan% , Potential: nan% ***

*** Average Profit: nan% , Potential: nan% ***
14237
*** Average Profit: nan% , Potential: nan% ***
14238
*** Average Profit: nan% , Potential: nan% ***
14239
*** Average Profit: nan% , Potential: nan% ***
14240
*** Average Profit: nan% , Potential: nan% ***
14241
*** Average Profit: nan% , Potential: nan% ***
14242
*** Average Profit: nan% , Potential: nan% ***
14243
*** Average Profit: nan% , Potential: nan% ***
14244
*** Average Profit: nan% , Potential: nan% ***
14245
*** Average Profit: nan% , Potential: nan% ***
14246
*** Average Profit: nan% , Potential: nan% ***
14247
*** Average Profit: nan% , Potential: nan% ***
14248
*** Average Profit: nan% , Potential: nan% ***
14249
*** Average Profit: nan% , Potential: nan% ***
14250
*** Average Profit: nan% , Potential: nan% ***
14251
*** Average Profit: nan% , Potential: nan% ***
14252
*** Average Profit: nan% , Potential: nan% ***
14253
*** Average Profit: nan% , Potential: nan% ***
14254
*** Average Profit: nan% , Potential: nan% ***

In [13]:
print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}

""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain),
    
))

NameError: name 'initial_investment' is not defined

# Shorting - current year 2019

In [14]:
report_short_all(2019)


**********- Score: -55.20000076293945 - STNG - NYSE: 39.34 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 15.19 - min: 13.02 - max: 39.69
Profit: 61.4 - max potential: 66.9 
Average growth rate: 0.887, consistent: 0, project growth: 0.094
DCF: 137.0, market_dcf: 28.7, adjusted_dcf: 28.6, dcf_to_price : -27.3



**********- Score: -45.599998474121094 - DCTH - NasdaqCM: 21.25 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 8.9 - min: 7.31 - max: 21.25
Profit: 58.1 - max potential: 65.6 
Average growth rate: -0.087, consistent: 0, project growth: -0.175
DCF: -33564.8, market_dcf: -34731.2, adjusted_dcf: -49036.5, dcf_to_price : -230860.0



**********- Score: -43.70000076293945 - UFCS - NasdaqGS: 43.73 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 27.51 - min: 23.2 - max: 46.73
Profit: 37.1 - max potential: 46.9 
Average growth rate: -0.828, consistent: 0, project growth: -1.393
DCF: -10169.1, market_dcf: -7

(24.073333333333327, 49.94666666666667)

In [15]:
report_short_all(2018)


**********- Score: -65.0 - TOPS - NasdaqCM: 16.462 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 0.8 - min: 0.6525 - max: 22.0
Profit: 95.1 - max potential: 96.0 
Average growth rate: 0.047, consistent: 0, project growth: 0.006
DCF: -19358.3, market_dcf: -24148.9, adjusted_dcf: -15825.7, dcf_to_price : -96234.73



**********- Score: -54.70000076293945 - INPX - NasdaqCM: 143.550143 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.905004 - min: 1.710001 - max: 193.500193
Profit: 96.6 - max potential: 98.8 
Average growth rate: -0.63, consistent: 0, project growth: -0.954
DCF: -596581.0, market_dcf: -81276.8, adjusted_dcf: -1476169.7, dcf_to_price : -1028430.36



**********- Score: -41.79999923706055 - SBOW - NYSE: 23.64 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 9.9 - min: 7.43 - max: 25.54
Profit: 58.1 - max potential: 68.6 
Average growth rate: -0.192, consistent: 0, project growth: -0.378
DCF: -60

(33.53333333333333, 54.236666666666665)

In [ ]:
report_short_all(2018)

In [147]:
!ipython nbconvert --to html Reporting_Rank1_2020_06_16.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Reporting_Rank1_2020_06_16.ipynb to html
[NbConvertApp] Writing 427042 bytes to Reporting_Rank1_2020_06_16.html
